#### import

In [219]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import get_effective_format
import pandas as pd

import re

import pygsheets
import pandas as pd

import google.auth
from googleapiclient.discovery import build
import pygsheets
#from google.oauth2.credentials import Credentials
from google.oauth2.service_account import Credentials

from typing import List, Optional
from abc import ABC, abstractmethod

import datetime
import re
from typing import Dict, List

#### consta

In [444]:
#CONST
KEY_JSON = 'google\\poised-backbone-191400-4e9fc454915f.json'

SHEET_NAME = 'Спонсорские ТНТ 27.07.23'
#SHEET_NAME = 'Copy of Спонсорские ТНТ 27.07.23 for script'
REPLACE_NAMES = {
    'Звезды в Африке': 'ЗВА',
    'Ярче звезд': 'ЯЗ',
    'Ярче Звезд': 'ЯЗ',
    'Шоу Воли': 'ШВ',
    'Битва Пикников': 'БП',
    'Лига городов': 'ЛГ',
    'Comedy Club': 'CC',
    'Женский StandUp': 'ЖS',
    'МОСКОВСКИЙ КАРТОФЕЛЬ': 'МОСКАР',
    'КАПИТАН ВКУСОВ': 'КАПИТАН',
}



SHEET_NAMES = {
    'tasks': 'ТАБЛИЧКА',
    'designers': 'Дизайнеры',
    'calendar': 'Календарь',
}

SHEET_INFO = {
    'spreadsheet_name': SHEET_NAME,
    'sheet_names': SHEET_NAMES,
}

FIELD_MAP = {
    "brand": "БРЕНД",
    "format_": "ФОРМАТ",
    "project_name": "ПРОЕКТ",
    "customer": "ЗАКАЗЧИК",
    "designer": "ДИЗАЙНЕР",
    "timing": "Тайминг",
    "status": "Статус",
    "color": "color",
    "color_status": "color_status",
    "task_name": "task_name",
    "task_id": "task_id"
}

COLOR_STATUS = {
       '#FFFFFF': 'work',
       '#808080': 'wait',
       '#CCCCCC': 'wait',
       '#B6D7A8': 'done',
       '#D9D1E9': 'pre_done'    
}

COLORS = {
    'white': '#FFFFFF',
    'dark_gray': '#909090',
    'med_gray': '#C0C0C0',
    'gray': '#E0E0E0',
    'light_gray': '#F0F0F0',
    'green': '#B6D7A8',
    'light_green': '#D9EAD3',
    'purple': '#D9D1E9',
    'black': '#000000'
}



### class

#### GoogleSheetInfo, Task

In [357]:
#1. Доменный слой
#Создадим основные сущности:

class Task:
    def __init__(self, brand, format_, project_name, customer, designer, timing, status, color, color_status, task_name, task_id):
        self.brand = brand
        self.format_ = format_
        self.project_name = project_name
        self.customer = customer
        self.designer = designer
        self.timing = timing
        self.status = status
        self.color = color
        self.color_status = color_status
        self.task_name = task_name
        self.task_id = task_id
    
    def __repr__(self):
        return self.task_name

#### GoogleSheetsTaskRepository

In [432]:

#2. Репозитории
#Определим интерфейсы и базовую реализацию:

class TaskRepository(ABC):
    ''' Репозиторий для работы с задачами'''
    @abstractmethod
    def get_all_tasks(self) -> List[Task]:
        pass


class GoogleSheetsTaskRepository(TaskRepository):
    ''' Репозиторий для работы с задачами в Google Таблицах'''        
    def __init__(self, sheet_info: GoogleSheetInfo, service: GoogleSheetsService):
        self.sheet_info = sheet_info
        self.service = service    
        self.df = None 
        self.replace_names = REPLACE_NAMES

    def get_all_tasks(self) -> List[Task]:
        ''' Получить все задачи'''
        self._check_df()
        projects = self._df_to_task(self.df)
        return projects

    def get_task_by(self, column_name, value):
        ''' Получить задачи по значению в колонке'''
        df = self._filter(column_name, value)
        return self._df_to_task(df)

    def get_task_by_id(self, project_id):
        return self.get_task_by('task_id', project_id)[0]

    def get_task_by_cstatus(self, color_status):
        ''' Получить задачи по цветовому статусу'''
        return self.get_task_by('color_status', color_status)

    def _load_and_process_data(self):
        ''' Загрузить и обработать данные из Google Таблицы'''
        df = self.service.get_dataframe(self.sheet_info.spreadsheet_name, self.sheet_info.get_sheet_name("tasks"))
        color_range = f'A1:A{len(df)}'
        colors = self.service.get_cell_colors(self.sheet_info.spreadsheet_name, self.sheet_info.get_sheet_name("tasks"), color_range)
        # в ячейке ДИЗАЙНЕР может быть несколько человек, там str в котором на каждой строчке один человек
        # надо эти строчки отсортировать по алфавиту удалив лишние пробелы в начале и конце каждоый строки
        # так же заменяем na на пустую строку        
        df['ДИЗАЙНЕР'] = df['ДИЗАЙНЕР'].fillna('').apply(lambda x: '\n'.join(sorted([i.strip() for i in x.split('\n') if i.strip()])))
        df['color'] = colors
        df['color_status'] = df['color'].apply(self._determine_status_from_color)
        df['task_id'] = df.index + 1
        df['task_name'] = df.apply(self._generate_task_name, axis=1)
        self.df = df

    def _generate_task_name(self, row):
        ''' Сгенерировать название задачи'''        
        format_ = row['ФОРМАТ'].split('\n')[0] if row['ФОРМАТ'] else ''
        name = row['БРЕНД'] + " [" + row['ПРОЕКТ'] + "] " + format_
        for key, value in self.replace_names.items():
            name = name.replace(key, value)
        return name

    def _check_df(self):
        ''' Проверить наличие данных'''
        if self.df is None:
            self._load_and_process_data()

    def _df_to_task(self, df):
        ''' Преобразовать DataFrame в список задач'''
        projects_list = []
        for idx, row in df.iterrows():
            project = {key: row[value] for key, value in FIELD_MAP.items()}
            project = Task(**project)
            projects_list.append(project)
        return projects_list
    
    def _filter(self, column_name, value):
        ''' Отфильтровать DataFrame по значению в колонке'''
        self._check_df()
        if not isinstance(value, list):
            value = [value]
        return self.df[self.df[column_name].isin(value)]
    
    def _determine_status_from_color(self, color):        
        ''' Определить статус по цвету'''
        cs = COLOR_STATUS
        return cs.get(color, "unknown")
    
    def _parse_timing(self, timing_str):
        ''' Разбор строки тайминга на даты и этапы '''
        lines = timing_str.split('\n')
        parsed_entries = []
        
        for line in lines:
            line = line.strip()  # Удаление лишних пробелов
            match = re.match(r"(\d{2}.\d{2})\s*[-:]*\s*(.+)", line)
            if match:
                date, task = match.groups()
                task = task.strip().rstrip('-').strip()
                parsed_entries.append((date, task))
        
        return parsed_entries
                   


#### GoogleSheetsService:

In [391]:
class GoogleSheetInfo:
    def __init__(self, spreadsheet_name, sheet_names):
        """
        Args:
        - spreadsheet_name (str): Название таблицы.
        - sheet_names (dict): Словарь с именами листов. Например, {"tasks": "ТАБЛИЧКА", "designers": "дизайнеры"}
        """
        self.spreadsheet_name = spreadsheet_name
        self.sheets = sheet_names

    def get_sheet_name(self, key):
        """Получить имя листа по ключу."""
        return self.sheets.get(key)

class GoogleSheetsService:
    ''' Сервис для работы с Google Таблицами'''
    def __init__(self, credentials_path):             
        credentials = Credentials.from_service_account_file(credentials_path)
        self.sheets_service = build('sheets', 'v4', credentials=credentials)
        self.drive_service = build('drive', 'v3', credentials=credentials)
        self.requests = []
        self.sheet_id_cache = {}
        self.get_spreadsheet_id_cache = {}

    def get_spreadsheet_id_by_name(self, spreadsheet_name):
        """Получение ID таблицы по ее имени."""        
        if spreadsheet_name in self.get_spreadsheet_id_cache:
            return self.get_spreadsheet_id_cache[spreadsheet_name]
        results = self.drive_service.files().list(q=f"name='{spreadsheet_name}'", fields="files(id, name)").execute()
        files = results.get('files', [])
        
        if not files:
            raise ValueError(f"Spreadsheet '{spreadsheet_name}' not found.")
        
        spreadsheet_id = files[0]['id']
        self.get_spreadsheet_id_cache[spreadsheet_name] = spreadsheet_id

        return spreadsheet_id

    def get_sheet_id_by_name(self, spreadsheet_name, worksheet_name):
        ''' Получить ID листа по имени '''
        cache_key = f"{spreadsheet_name}_{worksheet_name}"
        if cache_key in self.sheet_id_cache:
            return self.sheet_id_cache[cache_key]        
        print(f"Getting sheet id for {spreadsheet_name} {worksheet_name}")
        spreadsheet_id = self.get_spreadsheet_id_by_name(spreadsheet_name)
        result = self.sheets_service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
        for sheet in result['sheets']:
            if sheet["properties"]["title"] == worksheet_name:
                sheet_id = sheet["properties"]["sheetId"]
                self.sheet_id_cache[cache_key] = sheet_id
                return sheet_id

    def get_dataframe(self, spreadsheet_name, worksheet_name):
        '''Получить DataFrame по имени таблицы и имени листа.'''
        spreadsheet_id = self.get_spreadsheet_id_by_name(spreadsheet_name)
        result = self.sheets_service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id, 
            range=worksheet_name
        ).execute()
        values = result.get('values', [])
        df = pd.DataFrame(values[1:], columns=values[0])  # предполагая, что первая строка содержит заголовки
        return df
    
    def get_cell_colors(self, spreadsheet_name, worksheet_name, range_='A1:Z1000'):
        ''' Получить цвета ячеек листа'''        
        spreadsheet_id = self.get_spreadsheet_id_by_name(spreadsheet_name)
        
        result = self.sheets_service.spreadsheets().get(
            spreadsheetId=spreadsheet_id, 
            fields="sheets(data.rowData.values.userEnteredFormat.backgroundColor,properties.title)",
        ).execute()
        
        sheet = next(item for item in result['sheets'] if item["properties"]["title"] == worksheet_name)
        rows = sheet['data'][0].get('rowData', [])
    
        start_col, start_row, end_col, end_row = self._parse_range(range_)
    
        colors = []
        for r, row in enumerate(rows[start_row:end_row+1]):
            for c, cell in enumerate(row['values'][start_col:end_col+1]):
                color = cell['userEnteredFormat']['backgroundColor']
                colors.append(self._color_to_str(color))
    
        return colors
    
    def _color_to_str(self, color):
        ''' Конвертировать словарь цвета в строку формата #RRGGBB '''
        return "#{:02X}{:02X}{:02X}".format(int(color['red']*255), int(color['green']*255), int(color['blue']*255))
 
    def update_cell(self, spreadsheet_name: str, sheet_name: str, row: int, col: int, **kwargs):
        ''' Добавить запрос на обновление ячейки в пакет '''
        # запрещаем очишать лист с главной таблицей
        if sheet_name == SHEET_NAMES['tasks']:
            return
            
        fields = []
        cell_data = {}
        
        if "value" in kwargs:
            value = kwargs["value"]
            if pd.isna(value):
                value = ""
            cell_data["userEnteredValue"] = {"stringValue": value}
            fields.append("userEnteredValue")
        
        if "note" in kwargs:
            value = kwargs["note"]
            if pd.isna(value):
                value = ""
            cell_data["note"] = value
            fields.append("note")
        
        if "color" in kwargs:
            rgb = self._color_to_rgb(kwargs["color"])
            if isinstance(rgb, dict):
                cell_data["userEnteredFormat"] = {"backgroundColor": rgb}
                fields.append("userEnteredFormat.backgroundColor")            

        if "text_color" in kwargs:
            rgb = self._color_to_rgb(kwargs["text_color"])
            if isinstance(rgb, dict):
                cell_data["userEnteredFormat"] = {"textFormat": {"foregroundColor": rgb}}
                fields.append("userEnteredFormat.textFormat.foregroundColor")

        fields.append("userEnteredFormat.textFormat.foregroundColor")

        sheet_id = self.get_sheet_id_by_name(spreadsheet_name, sheet_name)
        
        request = [spreadsheet_name,{
            "updateCells": {
                "rows": {
                    "values": [cell_data]
                },
                "fields": ",".join(fields),
                "range": {
                    "sheetId": sheet_id,
                    "startRowIndex": row - 1,
                    "endRowIndex": row,
                    "startColumnIndex": col - 1,
                    "endColumnIndex": col
                }
            }
        }]
        self.requests.append(request)

    def _color_to_rgb(self, color):
        ''' Конвертировать строку hex в словарь rgb'''
        if isinstance(color, dict):
            rgb = color
        elif isinstance(color, str):
            color = color.lstrip('#')
            hlen = len(color)
            rgb = {"red": int(color[0:hlen//3], 16)/255, "green": int(color[hlen//3:2*hlen//3], 16)/255, "blue": int(color[2*hlen//3:hlen], 16)/255}
        elif isinstance(color, tuple) or isinstance(color, list):
            rgb = {"red": color[0], "green": color[1], "blue": color[2]}
        else:
            rgb = None
        return rgb

    def execute_updates(self, spreadsheet_name: str, requests: Optional[List] = None):
        ''' Выполнить пакетное обновление '''        
        if requests is None:
            req_list = [req[1] for req in self.requests if req[0] == spreadsheet_name]
        else:
            req_list = requests

        spreadsheet_id = self.get_spreadsheet_id_by_name(spreadsheet_name)

        try:
            batch_update_request = self.sheets_service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body={"requests": req_list})
            batch_update_request.execute()
            if requests is None:
                self.requests = [req for req in self.requests if req[0] != spreadsheet_name]
        except Exception as e:
            print(f"Error executing batch update: {e}")

    def clear_requests(self):
        ''' Очистить пакет обновлений '''
        self.requests = []

    def clear_cells(self, spreadsheet_name, sheet_name, range_='A1:BB1000'):
        ''' Очистить заданный диапазон ячеек от значений и заметок '''        
        # запрещаем очишать лист с главной таблицей
        if sheet_name == SHEET_NAMES['tasks']:
            return

        spreadsheet_id = self.get_spreadsheet_id_by_name(spreadsheet_name)
        sheet_id = self.get_sheet_id_by_name(spreadsheet_name, sheet_name)
        
        # Переводим диапазон в индексы
        start_col, start_row, end_col, end_row = self._parse_range(range_)
        
        request = {
            "updateCells": {
                "fields": "userEnteredValue,note",
                "range": {
                    "sheetId": sheet_id,
                    "startRowIndex": start_row,
                    "endRowIndex": end_row + 1,
                    "startColumnIndex": start_col,
                    "endColumnIndex": end_col + 1
                }
            }
        }
        
        self.execute_updates(spreadsheet_name, [request])
    
    def _parse_range(self, range_):
        ''' Конвертировать диапазон A1:Z1000 в индексы (0, 0, 25, 1000) 
        буквенные индексы могут быть из двех букв, например AA1:ZZ1000'''
        start, end = range_.split(':')
        start_col, start_row = self._cell_to_indices(start)
        end_col, end_row = self._cell_to_indices(end)
        return start_col, start_row, end_col, end_row

    def _cell_to_indices(self, cell):
        ''' Конвертировать AB1000 в (27, 999) '''
        col = 0
        row = 0
        for i in cell:
            if i.isalpha():
                col = col * 26 + ord(i) - 64
            else:
                row = row * 10 + int(i)
        return col-1, row-1



#### TaskManager

In [426]:
#3. Use Cases
#Реализуем основную логику:

class TaskManager:
    def __init__(self, task_repository):
        self.repository = task_repository
        self.tasks = None
        self.designers = None
    
    def update(self):
        ''' Обновить данные из репозитория'''
        self.tasks = self.repository.get_all_tasks()
        self.designers = set(task.designer for task in self.tasks if task.designer)

    def task_to_table(self, cstatus=['work', 'pre_done']):
        ''' Получить задачи в виде таблицы'''
        if cstatus == 'all':
            cstatus = ['work', 'pre_done', 'done']
        elif cstatus == 'done':
            cstatus = ['pre_done', 'done']
        elif isinstance(cstatus, str):
            cstatus = [cstatus]
        
        self.update()        
        tasks = [task for task in self.tasks if task.color_status in cstatus]
        # Получаем ID дизайнерского листа
        spreadsheet_name = self.repository.sheet_info.spreadsheet_name
        sheet_name = self.repository.sheet_info.get_sheet_name("designers")
        # Очищаем лист
        self.repository.service.clear_cells(spreadsheet_name, sheet_name)

        # Устанавливаем начальные координаты
        start_row, col = 2, 2
        row = start_row + 1

        for designer in sorted(self.designers):
            tasks_for_designer = [task for task in tasks if task.designer == designer]

            if not tasks_for_designer:
                continue

            # Добавляем имя дизайнера
            self.repository.service.update_cell(
                spreadsheet_name, sheet_name, start_row, col, value=designer, text_color=COLORS['white']
            )

            for task in tasks_for_designer:
                self.repository.service.update_cell(
                    spreadsheet_name, sheet_name, row, col, 
                    value=task.task_name, note=task.timing
                )
                row += 1

            col += 1
            row = start_row + 1

        self.repository.service.execute_updates(spreadsheet_name)



#### TaskTimingProcessor

In [427]:
from typing import List, Dict
import datetime

a = []

class TaskTimingProcessor:
    """ Класс для обработки тайминга задач """
    
    def __init__(self):
        self.date_pattern = re.compile(r"(\d{2}\.\d{2})")

    def parse_timing(self, timing_str: str) -> Dict[datetime.date, str]:
        """
        Преобразует строку тайминга в словарь, где ключи - даты, значения - этапы задач.
        """
        timings = {}
        
        # Разбиваем строку на отдельные строки по переносам
        lines = timing_str.strip().split("\n")
        
        for line in lines:
            line = line.strip()
            
            # Ищем дату в начале строки
            match = self.date_pattern.match(line)
            if not match:
                continue
            
            date_str = match.group(1)
            stage = line[len(date_str):].strip().strip('-').strip()
            
            # Преобразуем строку даты в объект datetime.date
            date_obj = datetime.datetime.strptime(date_str, "%d.%m").date().replace(year=datetime.datetime.now().year)
            
            timings[date_obj] = stage
        
        return timings
    
    def get_date_range(self, timings_dict: Dict[datetime.date, str]) -> (datetime.date, datetime.date):
        """ Возвращает минимальную и максимальную дату из словаря тайминга """
        min_date = min(timings_dict.keys())
        max_date = max(timings_dict.keys())
        return min_date, max_date

    def create_task_timing_structure(self, tasks: List[Task]) -> List[Dict]:
        """ 
        Создает структуру тайминга для задачи 
        """
        global_min_date = datetime.date.max
        global_max_date = datetime.date.min
        
        task_structure = []
        for task in tasks:
            timings_dict = self.parse_timing(task.timing)
            if not timings_dict:
                continue
            min_date, max_date = self.get_date_range(timings_dict)
            global_min_date = min(global_min_date, min_date)
            global_max_date = max(global_max_date, max_date)
            # если в стейдже есть слово "ответ" то не добавляем его в список
            timings_list = [{"date": date, "stage": stage} for date, stage in timings_dict.items() if "ответ" not in stage.lower()]
            task_structure.append({
                "id": task.task_id,
                "name": task.task_name,
                "designer": task.designer,
                "timings": timings_list,     
                "min_date": min_date,
                "max_date": max_date
            })
        return {
            'timings': task_structure,
            'min_date': global_min_date,
            'max_date': global_max_date
        }

#### CalendarManager

In [442]:
import pandas as pd
from collections import defaultdict
a = []

class CalendarManager:
    def __init__(self, sheet_info, service):
        self.sheet_info = sheet_info
        self.service = service
        self.calendar = None

    def create_calendar_structure(self, task_timings):
        calendar = defaultdict(lambda: defaultdict(list))
        for task in task_timings['timings']:
            designer = task['designer']
            for timing in task['timings']:
                date = timing['date']
                stage = task['name'] + ' [' + timing['stage'] + ']'
                calendar[date][designer].append(stage)
        return calendar

    def calendar_to_dataframe(self, calendar):
        # Преобразуем словарь в DataFrame
        df = pd.DataFrame(calendar).T
        df.index.name = 'Date'
        # Добавляем пропущенные даты
        min_date = df.index.min()
        max_date = df.index.max()
        full_date_range = pd.date_range(min_date, max_date)
        df = df.reindex(full_date_range).fillna("")
        # Преобразуем списки этапов в строки
        #df = df.applymap(lambda stages: "\n".join(stages) if stages else "")
        calendar = df.sort_index()
        self.calendar = calendar.reindex(sorted(calendar.columns), axis=1)
        return self.calendar

    def write_calendar_to_sheet(self, calendar, min_date='1W'):
        df = self.calendar_to_dataframe(calendar)
        if min_date:
            now = pd.Timestamp.now()
            min_date = now - pd.Timedelta(min_date)
            df = df[df.index >= min_date]
        spreadsheet_name = self.sheet_info.spreadsheet_name
        sheet_name = self.sheet_info.get_sheet_name("calendar")
    
        # Очищаем лист
        if sheet_name:
            self.service.clear_cells(spreadsheet_name, sheet_name)
        
        # Записываем имена дизайнеров (заголовок таблицы)
        for col_num, designer in enumerate(df.columns, start=1):
            self.service.update_cell(spreadsheet_name, sheet_name, 1, col_num+1, value=designer, text_color=COLORS['white'])
        
        # Записываем данные в Google Таблицу
        for row_num, (date, row) in enumerate(df.iterrows(), start=2):
            # Записываем дату слева от этапов
            if date.weekday() >= 5:  # проверяем на выходные дни                
                color = COLORS['med_gray']
            else:
                color = COLORS['gray']
            
            self.service.update_cell(spreadsheet_name, sheet_name, row_num, 1, value=date.strftime('%d.%m'), color=color,)
            now = pd.Timestamp.now().normalize()
            if date.weekday() >= 5:  # проверяем на выходные дни                
                color = COLORS['light_gray']
            else:
                color = COLORS['white']
            if date == now:                
                color = COLORS['light_green']
                
            if date < now:
                text_color = COLORS['dark_gray']
            else:
                text_color = None
            
            for col_num, stage in enumerate(row, start=2):
                if isinstance(stage, list):
                    value = "\n".join(stage) if stage else ""
                elif pd.isna(stage):
                    value = ""
                else:
                    value = stage
    
                self.service.update_cell(spreadsheet_name, sheet_name, row_num, col_num, value=value, color=color, text_color=text_color)
                    
        self.service.execute_updates(spreadsheet_name)

#### GoogleSheetPlanner

In [429]:
class GoogleSheetPlanner:
    def __init__(self, key_json, sheet_info_data):
        self.sheet_info = GoogleSheetInfo(**sheet_info_data)
        self.sheets_service = GoogleSheetsService(key_json)
        self.timing_processor = TaskTimingProcessor()
        self.task_repository = GoogleSheetsTaskRepository(self.sheet_info, self.sheets_service)
        self.task_manager = TaskManager(self.task_repository)
        self.calendar_manager = CalendarManager(self.sheet_info, self.sheets_service)
        
    def task_to_table(self, cstatus=['work', 'pre_done']):
        return self.task_manager.task_to_table(cstatus)

    def task_to_calendar(self, cstatus=['work', 'pre_done'], min_date='1W'):
        tasks = planner.task_repository.get_task_by_cstatus(cstatus)
        task_timings = self.timing_processor.create_task_timing_structure(tasks)
        calendar = self.calendar_manager.create_calendar_structure(task_timings)
        self.calendar_manager.write_calendar_to_sheet(calendar, min_date)
    
    def update(self):
        return self.task_manager.update()

### test

In [445]:
planner = GoogleSheetPlanner(KEY_JSON, SHEET_INFO)
planner.update()
planner.task_to_calendar()
planner.task_to_table()

Getting sheet id for Спонсорские ТНТ 27.07.23 Календарь
Getting sheet id for Спонсорские ТНТ 27.07.23 Дизайнеры


In [384]:
planner = GoogleSheetPlanner(KEY_JSON, SHEET_INFO)
tasks = planner.task_repository.get_task_by_cstatus('work')
timing = planner.timing_processor.create_task_timing_structure(tasks)
calendar = planner.calendar_manager.create_calendar_structure(timing)
df = planner.calendar_manager.calendar_to_dataframe(calendar)
planner.calendar_manager.write_calendar_to_sheet(calendar)

Getting sheet id for Спонсорские ТНТ 27.07.23 Календарь


In [302]:
a = []

In [ ]:
a

In [68]:
sheet_info = GoogleSheetInfo(**SHEET_INFO)
sheets_service = GoogleSheetsService(KEY_JSON)
tp = TaskTimingProcessor()
projects = GoogleSheetsTaskRepository(sheet_info, sheets_service, tp)
task_manager = GoogleSheetsTaskManager(projects)
task_manager.task_to_table(['work', 'pre_done'])

In [30]:


timeing = '''
"15.09 раскадровка
18.09 ответ клиента
19.09 анимация
20.09 сдача в продакшн
20.09 - сдача до 14:00
период размещения- ноябрь - декабрь
Съемки ярче звезд 4 октября"
'''

projects._parse_timing(timeing)

[('18.09', 'ответ клиента'),
 ('19.09', 'анимация'),
 ('20.09', 'сдача в продакшн'),
 ('20.09', 'сдача до 14:00')]